# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

In [7]:
data.shape

(4870, 65)

### Tests

- As this is a data with dependent variable having binary classification, Chi-Squared test of independence would be appropriate for this problem.

- The necessary assumptions for Central limit theoram are :

   1. The sample values must be independent of each other. This means that the occurrence of one event has no influence on the next event. Usually, if we know that people or items were selected randomly we can assume that the independence assumption is met.
   
   2. The sample size must be more than 30 in general.
   
The observations in the dataset are independent of any other observation made. In other words, no observation affects any other observation in our given dataframe.

The races are qualitative in nature; they are either black or white. Similarly, the callbacks are also qualitative in nature; they are either a success or a failure. Therefore, we are trying to check for correlation between two qualitative features. So central limit theroam also be applied to this problem.


### Null and Alternative Hypothesis

Null Hypothesis : Race and Callbacks are independent of eachother.

Alternative Hypothesis: Race and Callbacks have a relationship.

### Margin of error, confidence interval, and p-value

In [15]:
aggfunc = {
    'id': 'count',
}

callback_df = data.pivot_table('id', ['call'], 'race', aggfunc=aggfunc)
callback_df.index = ['Failure', 'Success']
callback_df.columns = ['Black', 'White']

callback_df.loc['Total'] = [ callback_df['Black'].sum(), callback_df['White'].sum()]
callback_df['Total'] = callback_df['Black'] + callback_df['White']

callback_df

,Black,White,Total
Failure,2278,2200,4478
Success,157,235,392
Total,2435,2435,4870


In [16]:
total = callback_df.loc['Total', 'Total']

In [18]:
group_total = callback_df.loc['Total', 'Black']

In [19]:
group_total

2435

In [20]:
#Likelihood of getting and not getting a callback for the entire sample
p1 = callback_df.loc['Success', 'Total']/total
p0 = callback_df.loc['Failure', 'Total']/total

p0, p1

(0.9195071868583162, 0.08049281314168377)

In [21]:
estimate_array = np.array([[p1, p1], [p0, p0]])
group_totals = np.array([[group_total], [group_total]]) #It is the same for both races in this case

estimate_array = np.array(estimate_array * group_totals)

estimate_array

array([[ 196.,  196.],
       [2239., 2239.]])

Excluding the Total row and column, there are 2 rows and 2 columns in our Callback DataFrame. Therefore, the **degree of freedom is 1.**

In [22]:
#actual_array = np.array([], [])
success_array = np.array(callback_df.loc['Success', ['Black', 'White']])
failure_array = np.array(callback_df.loc['Failure', ['Black', 'White']])

actual_array = np.array([success_array, failure_array])
actual_array

array([[ 157,  235],
       [2278, 2200]], dtype=int64)

In [23]:
chi2 = (estimate_array - actual_array) ** 2 / estimate_array
chi2 = np.sum(chi2)

chi2

16.879050414270225

In [24]:
df = 1
p = 1 - stats.chi2.cdf(chi2, df)
p

3.983886837588546e-05

The p-value obtained is extremely small as compared to the threshold $\alpha$ of 0.1. This implies that we have to reject the null hypothesis. In other words, **there is a clear correlation between the race of a candidate and the success in getting a callback based on resume.**

Since the significance level, $\alpha$ is 10%, this directly implies that the confidence level is 90%.

## Significance of the Difference (Z-Test)

Based on the chi square significance test performed above, it seems pretty clear that there is a correlation between race and successful callbacks.

One other statistic that we can compute is if the difference in success is statistically significant or not. If it is, then we can unanimously conclude that there is racial discrimimation taking place in the industry.

In [26]:
white_mean = callback_df.loc['Success', 'White']/callback_df.loc['Total', 'White']
white_std = callback_df.loc['Success', 'White'] * ((1 - white_mean) ** 2) + callback_df.loc['Failure', 'White'] * ((0 - white_mean) ** 2)
white_std = np.sqrt(white_std/group_total)

white_mean, white_std

(0.09650924024640657, 0.29528834517039093)

In [27]:
black_mean = callback_df.loc['Success', 'Black']/callback_df.loc['Total', 'Black']
black_std = callback_df.loc['Success', 'Black'] * ((1 - black_mean) ** 2) + callback_df.loc['Failure', 'Black'] * ((0 - black_mean) ** 2)
black_std = np.sqrt(black_std/group_total)

black_mean, black_std

(0.06447638603696099, 0.24559963697158382)

### Hypothesis Testing

The following hypotheses are defined:

* **Null Hypothesis**: There is no difference in mean of callbacks for blacks and whites.
* **Alternate Hypothesis**: There is a difference in mean of callbacks for blacks and whites

Assuming $\alpha$ to be 0.1 and the null hypothesis to be true.

In [28]:
h0_mean = 0

mean_diff = white_mean - black_mean
#sigma_diff = np.sqrt((white_std**2)/group_total  + (black_std**2)/group_total)
p_hat = (callback_df.loc['Success', 'Black'] + callback_df.loc['Success', 'White'])/(callback_df.loc['Total', 'Black'] + callback_df.loc['Total', 'White']) 
var_diff = p_hat * (1- p_hat) * (1/callback_df.loc['Total', 'Black'] + 1/callback_df.loc['Total', 'White'])
sigma_diff = np.sqrt(var_diff)

mean_diff, sigma_diff

(0.032032854209445585, 0.007796894036170457)

In [29]:
z = (mean_diff - h0_mean) / sigma_diff
z

4.108412152434346

In [30]:
p = (1-stats.norm.cdf(z))*2
p

3.983886837577444e-05

In [31]:
error = 1.96 * sigma_diff
error

0.015281912310894095

In [32]:
range_upper = mean_diff + error
range_lower = mean_diff - error
confidence_interval = range_lower, range_upper

confidence_interval

(0.01675094189855149, 0.04731476652033968)

The p-value obtained is much lower than the significance level and almost identical to the p-value obtained in the chi square significance test. Therefore, we can safely reject the null hypothesis. This test further strengthens our initial claim of a correlation existing between race and callbacks.

The margin of error is 1.5% for a confidence level of 95% and a z-value of 1.96. The confidence interval at this level is (0.016, 0.047).

## Conclusion and Final Remarks

1. There is a clear correlation between the race and the callback success of a particular person. This could imply that there is active discrimination taking place in the industry based on race.
2. However, we cannot conclude that race is the most important factor for callback success. While a correlation has been established, this does not directly imply causation. Other parameters such as education and work experience may also have a role to play and the relationship between all these variables have not been established to arrive at a definitive conclusion.